In [127]:
from flask import Flask, request, Blueprint
import pymongo
import numpy as np
import json
from bson import ObjectId
import base64
from pymongo.database import Database 
from pyisemail import is_email
import smtplib
from email.mime.text import MIMEText
from email.utils import formataddr
from random import randint
from operator import itemgetter
from collections import Counter
import matplotlib.pyplot as plt
from io import BytesIO
import os
from PIL import Image

In [128]:
def mongodb_init():
    # connect to mongodb
    mongo = pymongo.MongoClient(host='18.162.214.19', port=27017, username="root", password="1647#4hkust",
                                authSource='admin')
    print('数据库当前的databases: ', mongo.list_database_names())
    return mongo


def get_db(mongo, db_name):
    db = Database(name=db_name, client=mongo)
    print('获取/创建库：', db.name)
    return db



# API below
app = Flask(__name__)
mongo = mongodb_init()
db = get_db(mongo, 'chen_db')

数据库当前的databases:  ['admin', 'chen_db', 'config', 'local']
获取/创建库： chen_db


In [129]:
def compress_image(infile, outfile='', mb=30, step=15, quality=60):
    """
    不改变图片尺寸压缩到指定大小
    :param infile: 压缩源文件
    :param outfile: 压缩文件保存地址
    :param mb: 压缩比率
    :param step: 每次调整的压缩比率
    :param quality: 初始压缩比率
    :return: 压缩文件地址，压缩文件大小
    """
    o_size = os.path.getsize(infile) / 1024
    print(f"original size: {o_size:.2f} KB")
    if o_size <= 5:
        return infile
    outfile = get_outfile(infile, outfile)
    
    while o_size > o_size * mb/100:
        im = Image.open(infile)
        im.save(outfile, quality=quality)
        if quality - step < 0:
            break
        quality -= step
        o_size = os.path.getsize(outfile) / 1024
        
    return outfile, o_size

def get_outfile(infile, outfile):
    if outfile:
        return outfile
    dir, suffix = os.path.splitext(infile)
    outfile = '{}-out{}'.format(dir, suffix)
    return outfile

In [130]:
def Image_Encode(image_URL):
    # 压缩图片
    compressed_image, compressed_size = compress_image(infile = image_URL)
    # 读取本地图片文件
    print(f"Size after compress: {compressed_size:.2f} KB")
    
    with open(compressed_image, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据

    # 将二进制图片编码为 base64 格式
    base64_data_bytes = base64.b64encode(image_data)  # 图片的 bytes 类型
    base64_data_str = base64_data_bytes.decode()  # 图片的字符串类型\
    return base64_data_str

In [131]:
def Image_show(image_URL):
    with open(image_URL, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据
    # 展示图片
    img_data = plt.imread(BytesIO(image_data), format="JPG")
    import cv2
    img_cv2 = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
    cv2.imshow('Image', img_cv2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [132]:
# set the image directories
imageUrl_1 = '../image/' + '1.webp'
imageUrl_2 = '../image/' + '2.webp'
imageUrl_3 = '../image/' + '3.webp'
imageUrl_4 = '../image/' + '4.webp'
Images = []
Images.append( Image_Encode(imageUrl_1) )
Images.append( Image_Encode(imageUrl_2) )
Images.append( Image_Encode(imageUrl_3) )
# Images.append( Image_Encode(imageUrl_4) )

# set parameters
Title = "Bmx bicycle."

Text = \
"""
Selling a movelo bmx bicycle for kids. Good condition. 18x2.25 tires.
"""

Price = "80"

original size: 18.60 KB
Size after compress: 8.02 KB
original size: 24.92 KB
Size after compress: 10.84 KB
original size: 16.71 KB
Size after compress: 6.42 KB


In [133]:
imageUrl_checks = '../image/' + '1-out.webp'
# Image_show(imageUrl_1)
# Image_show(imageUrl_checks)

In [137]:
import requests
url = 'http://localhost:8080/new-post'
body = {
            'PostOwner': '1030920919@qq.com',
            'Title': Title,
            'Text': Text,
            'Auction': False,
            'LostFound': False,
            'Price': Price,
            'Fields': [],
            'Images': Images
           }
r = requests.post(url, json=body)
print(r.text)

{
  "Success": true
}

